# SpotTheDog Model

In [ ]:
import os # for file path
import requests # for downloading image from firebase storage url
import cv2 # for image
import dlib # for predictor and detector
import face_recognition # for face encoding and recognizing
import firebase_admin # firebase database link
from firebase_admin import credentials, initialize_app, storage, firestore

# 1. Initialization

In [ ]:
DATA_PATH = './data/'

face_landmark_detector_file = os.path.join(DATA_PATH, 'dogHeadDetector.dat')
face_landmark_predictor_file = os.path.join(DATA_PATH, 'landmarkDetector.dat')

detector = dlib.cnn_face_detection_model_v1(face_landmark_detector_file)
predictor = dlib.shape_predictor(face_landmark_predictor_file)

### 1.1 Connecting to Database

In [ ]:
def browse_db(collection_name, documentID, field_name):
    """
        Browse database - 
            parameter: 
                collection_name (str):
                    collection that the function should get the value of it
                field_name (str): 
                        field name that the function should return the value of it
            return: res (list):
                    list that field_name contain from database
    """
    # key_path = os.path.join(DATA_PATH, 'spot-the-dog-e68bd-f5c41104fb22.json')
    # cred = credentials.Certificate(key_path)  
    # firebase_admin.initialize_app(cred)

    db = firestore.client()     # connecting to firestore 

    if collection_name == "mlCollection" and documentID == None:
        ml_collection = db.collection(collection_name)
        res = ml_collection.document('70Ey6ANXgA51mSP7bSe4').get().to_dict()[field_name]
    elif collection_name != "mlCollection":
        collection = db.collection(collection_name).orderBy("timestamp", "desc").limit(1).get()
        res = collection.document(documentID).to_dict()[field_name]
    return res

def update_db(collection_name, documentID, field_name, data):
    """
    update database - 
        parameter: 
            collection_name (str):
                    collection that the function should update the value of it
            field_name (str): 
                    field name that the function should update the value of it
            data (str / list):
                    data to be updated
        return: None
    """
    # key_path = os.path.join(DATA_PATH, 'spot-the-dog-e68bd-f5c41104fb22.json')
    # cred = credentials.Certificate(key_path)  
    # firebase_admin.initialize_app(cred)
    
    db = firestore.client()     # connecting to firestore 

    # if updating 'reporter' database collection
    if collection_name == 'reporter' and documentID != None:
        reporter_collection = db.collection('reporter')

        if field_name == "face_encoding" or field_name == "matched_names":
            res = reporter_collection.document(documentID).get().to_dict()[fieldname]
            res.append(data)

        res = reporter_collection.document(documentID).update({field_name:res})
        
    # if updating 'mlCollection' database collection
    elif collection_name == 'mlCollection'and documentID == None:
        ml_collection = db.collection('mlCollection')
        res = ml_collection.document('70Ey6ANXgA51mSP7bSe4').get().to_dict()[field_name]
        res.append(data)
        res = ml_collection.document('70Ey6ANXgA51mSP7bSe4').update({field_name:res})


def read_url_image(url, name):
    """
    Download Image from URL in Database -
        parameters:
            url:
                image URL in Database
            name (str):
                document ID
    """
    # image url from database
    r = requests.get(url=url)

    # temporary image save format
    TEMPORARY_IMAGE_PATH = "./img_temp/"
    FILE_NAME = name + ".jpg"
    FILE_PATH = os.path.join(TEMPORARY_IMAGE_PATH, FILE_NAME)

    # save temporary image
    if r.status_code == 200:
        print("File Downloaded")
        with open(FILE_PATH, 'wb') as f:
            f.write(r.content)
            print("File Saved")
    else:
        raise Exception("Something went wrong")
    
    # read image using cv2
    image = cv2.imread(FILE_PATH)
    
    # delete image
    os.remove(FILE_PATH)
    print("File Removed")
    
    return image

# 2. Functions

## 2.2. <code>face_recognition</code> Function Modifications

In [ ]:
def _trim_css_to_bounds(css, image_shape):
    return max(css[0], 0), min(css[1], image_shape[1]), min(css[2], image_shape[0]), max(css[3], 0)


def _rect_to_css(rect):
    return rect.top(), rect.right(), rect.bottom(), rect.left()


def _raw_face_locations(img, number_of_times_to_upsample=1):
    return detector(img, number_of_times_to_upsample)


def face_locations(img, number_of_times_to_upsample=1):
    return [_trim_css_to_bounds(_rect_to_css(face.rect), img.shape) for face in _raw_face_locations(img, number_of_times_to_upsample)]

## 2.3. Main Functions

In [ ]:
def draw_label(input_image, coordinates, label):
    labeled_image = input_image.copy()
    
    (top, right, bottom, left) = coordinates
    cv2.rectangle(labeled_image, (left, top), (right, bottom), (0, 255, 0), 5)
    cv2.putText(labeled_image, label, (left - 10, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    
    return labeled_image


def detect_dog_face(input_image):
    detected_image = input_image.copy() # generate safe copy
    
    gray_image = cv2.cvtColor(detected_image, cv2.COLOR_BGR2GRAY) # convert color channel from BGR to gray scale
    dets_locations = face_locations(gray_image, 1) # detect faces
    
    return dets_locations


def match_face(face_encoding, registered_face_encodings, registered_face_names, tolerance=0.4):
    matched_names = [] # initialization
    
    matches = face_recognition.compare_faces(registered_face_encodings, face_encoding, tolerance) # list of true or false
    # face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    
    index = [i for i, val in enumerate(matches) if val] # find index list where matches are true
    matched_names = [registered_face_names[i] for i in index] # find id numbers from index list
    
    return matched_names

# 3. Operation

## 3.1. Reporter Side

### 3.1.1. Inputs from User Prompt

In [ ]:
document = {
    'did': "0001",
    'image': "https://firebasestorage.googleapis.com/v0/b/kreaters-b978a.appspot.com/o/D72-C607-4452-BCF9-B71A4C4554DD.png?alt=media&token=2f23610b-fb92-4670-a727-f11a68a066d8"
}

In [ ]:
name = document.get('did')
input_image_url = document.get('image')
input_image = read_url_image(input_image_url, name)

### 3.1.2. Get <code>face_encoding</code>

In [ ]:
dets_locations = detect_dog_face(input_image)

# filter image with only single dog face
if len(dets_locations) == 0:
    raise Exception("No face has been detected.")
elif len(dets_locations) > 1:
    raise Exception("More than one face have been detected.")
        
face_encoding = face_recognition.face_encodings(input_image, dets_locations)[0] # array

3.1.3. Update Collections in Database

In [ ]:
# ML Collection 
update_db("mlCollection", None, "reported_face_encodings", face_encoding)
update_db("mlCollection", None, "reported_face_names", name)

## 3.2. Owner Side

### 3.2.1. Inputs from User Prompt

In [ ]:
document = {
    'did': "0001",
    'image': "https://firebasestorage.googleapis.com/v0/b/kreaters-b978a.appspot.com/o/D72-C607-4452-BCF9-B71A4C4554DD.png?alt=media&token=2f23610b-fb92-4670-a727-f11a68a066d8",
    'face_encoding': [],
    'matched_names': []
}

In [ ]:
name = document.get('did')
input_image_url = document.get('image')
input_image = read_url_image(input_image_url, name)


### 3.2.2. Get <code>face_encoding</code>

In [ ]:
if len(document.get("face_encoding")) == 0:
    # detect dog faces
    dets_locations = detect_dog_face(input_image)

    # filter image with only single dog face
    if len(dets_locations) == 0:
        raise Exception("No face has been detected.")
    elif len(dets_locations) > 1:
        raise Exception("More than one face have been detected.")
        
face_encoding = face_recognition.face_encodings(input_image, dets_locations)[0] # array

### 3.2.3. Browse data from mlCollection

In [ ]:
reported_face_encodings = browse_db("mlCollection", None, "reported_face_encodings")
reported_face_names = browse_db("mlCollection", None, "reported_face_names")

### 3.2.4. Match Faces

In [ ]:
if len(reported_face_encodings) == 0 or len(reported_face_names) == 0:
    raise Exception("There is currently no reported image.")
    
matched_names = match_face(face_encoding, reported_face_encodings, reported_face_names)

### 3.2.5. Update Owner Collection

In [ ]:
document['face_encoding'] = face_encoding
document['matched_names'] = matched_names

In [ ]:
update_db("owner", document.get("name"), "face_encoding", face_encoding)
update_db("owner", document.get("name"), "matched_names", matched_names)

## 3.3. <code>operation_function.py</code>

In [ ]:
import os
import cv2
import face_recognition
import spot_the_dog_model


def reporter(document):
    """
    reporter side call function -
        parameters:
            document (dict):
                meta data followed by {
                    'did': id_number,
                    'image': image_uri
                    }
    
    This function takes an image from reporter's submission and stor its face encoding to mlCollection.
    """


    """Initialization"""
    name = document.get('did') # id
    image_url = document.get('image')
    input_image = spot_the_dog_model.read_url_image(image_url, name)


    """Detect Dog Face and Obtain face_encoding"""
    dets_locations = spot_the_dog_model.detect_dog_face(input_image)

    # filter image with only single dog face
    if len(dets_locations) == 0:
        raise Exception("No face has been detected.")
    elif len(dets_locations) > 1:
        raise Exception("More than one face have been detected.")
            
    face_encoding = face_recognition.face_encodings(input_image, dets_locations)[0] # face_enconding array


    """Update mlCollection"""
    # ML Collection 
    spot_the_dog_model.update_db("mlCollection", None, "reported_face_encodings", face_encoding)
    spot_the_dog_model.update_db("mlCollection", None, "reported_face_names", name)


def owner(document):
    """
    owner side call function - 
        parameters:
            document (dict):
                meta data followed by {
                    'did': id_number,
                    'image': image_uri,
                    'face_encoding': np.array,
                    'matched_names': string_list
                    }

    This function takes an image from owner's submission and find similar dogs from mlCollections.
    """


    """Initialization"""
    name = document.get('did') # id
    image_url = document.get('image')
    input_image = spot_the_dog_model.read_url_image(image_url, name)


    """Detect Dog Face and Obtain face_encoding"""
    if len(document.get("face_encoding")) == 0: # check if the document has face_encoding already
        # detect dog faces
        dets_locations = spot_the_dog_model.detect_dog_face(input_image)

        # filter image with only single dog face
        if len(dets_locations) == 0:
            raise Exception("No face has been detected.")
        elif len(dets_locations) > 1:
            raise Exception("More than one face have been detected.")
            
    face_encoding = face_recognition.face_encodings(input_image, dets_locations)[0] # face_enconding array

    """Browse Data From mlCollection"""
    reported_face_encodings = spot_the_dog_model.browse_db("mlCollection", None, "reported_face_encodings")
    reported_face_names = spot_the_dog_model.browse_db("mlCollection", None, "reported_face_names")


    """Match Faces"""
    if len(reported_face_encodings) == 0 or len(reported_face_names) == 0: # check if there is no reported dog
        raise Exception("There is currently no reported image.")
    
    matched_names = spot_the_dog_model.match_face(face_encoding, reported_face_encodings, reported_face_names)


    """Update ownerCollection"""
    spot_the_dog_model.update_db("owner", document.get("did"), "face_encoding", face_encoding)
    spot_the_dog_model.update_db("owner", document.get("did"), "matched_names", matched_names)

# 4. References